In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split # for #2
from sklearn.model_selection import cross_val_score # for #3
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn import svm
import matplotlib.pyplot as plt
import warnings
import os
%matplotlib inline

warnings.filterwarnings("ignore") #, category=DeprecationWarning)

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [3]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

# app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # silence the deprecation warning

# db = SQLAlchemy(app)

In [4]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [5]:
# TODO Fix SQL input/output; temp fix:
df = pd.read_pickle('data/kickstarter_data.pkl')

In [6]:
print(df.shape)
df.head()

(19291, 20)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved,main_category_code,country_code,currency_code,state_code
ID,,,,,,,,,,,,,,,,,,,,
1031685482,You Sparkle Inside: A Kids' Book + Commissions...,Children's Books,Publishing,USD,2018-01-02,2000.0,2017-12-06,6083.00,successful,133,US,1054.0,6083.00,2000.0,27 days,304.150000,12,21,13,1
1176736360,THE NEW BILLY WYLDER RECORD,Indie Rock,Music,USD,2018-01-02,10000.0,2017-11-30,11169.56,successful,177,US,650.0,11169.56,10000.0,33 days,111.695600,10,21,13,1
1894703097,Driftwood Holly ***The Venice Project***,Indie Rock,Music,EUR,2018-01-02,25000.0,2017-11-28,25417.20,successful,168,DE,0.0,30615.02,30112.5,35 days,101.668809,10,5,4,1
311863059,On a Limb's debut album,Jazz,Music,USD,2018-01-02,1000.0,2017-12-09,1743.00,successful,39,US,0.0,1743.00,1000.0,24 days,174.300000,10,21,13,1
2076121120,Briggs Barbecue Sauces,Small Batch,Food,USD,2018-01-02,200000.0,2017-11-03,1.00,failed,1,US,0.0,1.00,200000.0,60 days,0.000500,7,21,13,0


In [7]:
X = df[['main_category_code','country_code','currency_code','usd_goal_real']]
y = df['state_code']

k_range = list(range(1, 101))
print(X.shape)
print(y.shape)

(19291, 4)
(19291,)


## SVM

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=42)

In [9]:
pd.Series(y_train).value_counts()

0    5460
1    4185
Name: state_code, dtype: int64

## Principal Components Analysis

In [10]:
pca = PCA(n_components=2)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [11]:
pcafeatures_train = pca.transform(X_train)

In [12]:
# from itertools import cycle

# def plot_PCA_2D(data, target, target_names):
#     colors = cycle(['r','g','b','c','m','y','orange','w','aqua','yellow'])
#     target_ids = range(len(target_names))
#     plt.figure()
#     fig, ax = plt.subplots(figsize=(20, 7))

#     for i, c, label in zip(target_ids, colors, target_names):
#         plt.scatter(data[target == i, 0], data[target == i, 1],
#                    c=c, label=label)
#     plt.legend()

In [ ]:
# plot_PCA_2D(pcafeatures_train, target=y_train, target_names=state_code)

## Fitting Linear and RBF SVM Models

In [ ]:
model_svm = svm.SVC(kernel='linear')
model_svm.fit(X_train, y_train)

In [ ]:
y_pred = model_svm.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
model_svm2 = svm.SVC(kernel='rbf', gamma = 0.001)
model_svm2.fit(X_train, y_train)

In [ ]:
y_pred2 = model_svm2.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred2)

In [ ]:
confusion_matrix(y_test,y_pred2)